In [1]:
import parmed
import numpy as np
import itertools

from pdbfixer import PDBFixer
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from rdkit import Chem
from rdkit.Chem import rdmolfiles
from rdkit.Chem import AllChem

from sys import stdout

from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField
from pkg_resources import iter_entry_points

you got numpy


In [2]:
import simtk.openmm

In [3]:
for entry_point in iter_entry_points(group='openforcefield.smirnoff_forcefield_directory'):
     print(entry_point.load()())
        
ff = ForceField('openff_unconstrained-1.0.0.offxml')

['/home/lewis/.cache/Python-Eggs/openforcefields-1.0.0-py3.7.egg-tmp/openforcefields/offxml']
['/home/lewis/.cache/Python-Eggs/smirnoff99frosst-1.1.0-py3.7.egg-tmp/smirnoff99frosst/offxml']


In [4]:
topology.Topology.loadBondDefinitions('../data/dppc.xml')
prot_pdb = PDBFile('../data/charmm-gui/step5_assembly.pdb')
omm_forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml', 'amber14/lipid17.xml')
prot_system = omm_forcefield.createSystem(prot_pdb.topology, rigidWater=False)
prot_structure = parmed.openmm.load_topology(prot_pdb.topology,
                                           prot_system,
                                           xyz=prot_pdb.positions)

In [5]:
mol = Chem.MolFromSmiles('CCCCCC1=CC(=C(C(=C1)O)[C@@H]2C=C(CC[C@H]2C(=C)C)C)O')
mol=Chem.AddHs(mol)
AllChem.EmbedMolecule(mol)
Chem.rdmolfiles.MolToPDBFile(mol, './cbd.pdb')

drug_pdbfile = PDBFile('./cbd.pdb')
drug = Molecule.from_smiles('CCCCCC1=CC(=C(C(=C1)O)[C@@H]2C=C(CC[C@H]2C(=C)C)C)O')

off_topology = Topology.from_openmm(openmm_topology=drug_pdbfile.topology,
                                   unique_molecules=[drug])

In [6]:
drug_system = ff.create_openmm_system(off_topology)


In [ ]:
##Writing an XML file.

In [8]:
with open('drug_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                drug_system
            )
    )

In [37]:
drug_structure = parmed.openmm.load_topology(drug_pdbfile.topology,
                                                drug_system,
                                                xyz=drug_pdbfile.positions)

In [38]:
drug_structure.coordinates = drug_structure.coordinates-np.mean(drug_structure.coordinates, axis=0)
drug_structure.coordinates = drug_structure.coordinates+ np.array([np.max(prot_structure.coordinates[:,0])+10,
                                                                         np.min(prot_structure.coordinates[:,1])+10,
                                                                         0])


In [39]:
complex_structure = prot_structure + drug_structure
for _ in range(6):
    drug_structure.coordinates = drug_structure.coordinates+ np.array([0,10,0])
    complex_structure = complex_structure+drug_structure

In [40]:
drug_structure.coordinates = drug_structure.coordinates-np.mean(drug_structure.coordinates, axis=0)
drug_structure.coordinates = drug_structure.coordinates+ np.array([np.min(complex_structure.coordinates[:,0]),
                                                                         np.max(prot_structure.coordinates[:,1])+10,
                                                                         0])

In [41]:
for _ in range(6):
    drug_structure.coordinates = drug_structure.coordinates+ np.array([16,0,0])
    complex_structure = complex_structure+drug_structure

In [42]:
#translate box to 0,0,0. Do a visual check on the test.pdb to make sure its going correctly. 
complex_structure.coordinates = complex_structure.coordinates-np.min(complex_structure.coordinates, axis=0)
complex_structure.save('./test.pdb', overwrite=True)

newbox = (np.max(complex_structure.coordinates,axis=0)+np.array(2))*angstroms
complex_structure.box = (newbox[0], newbox[0], newbox[2], 90, 90, 90)
print(newbox)
print(complex_structure.box)

[105.88        96.13196226 103.007     ] A
[105.88  105.88  103.007  90.     90.     90.   ]


In [43]:
proteinAtoms = [count for count, i in enumerate(complex_structure.topology.atoms()) if i.residue.name not in ["DPPC", "CL", "NA", "HOH", "UNL"]]

In [44]:
complex_structure.coordinates = complex_structure.coordinates-np.mean(complex_structure.coordinates[proteinAtoms], axis=0)+np.array([3,3,3])

In [51]:
complex_system = complex_structure.createSystem(nonbondedMethod=PME,
                                                nonbondedCutoff=0.9*nanometer,
                                               constraints=HBonds,
                                                rigidWater=True)

In [52]:
force1 = CustomExternalForce("k1*periodicdistance(x, y, z, x0, y0, z0)^2")
force1.addGlobalParameter("k1", 0.1*kilocalories_per_mole/angstroms**2)
force1.addPerParticleParameter("x0")
force1.addPerParticleParameter("y0")
force1.addPerParticleParameter("z0")
for i, z in zip(prot_pdb.topology.atoms(), prot_pdb.getPositions()):                  
    if i.name=='CA':
        force1.addParticle(int(i.index), [z[0], z[1], z[2]])
        
print(force1.usesPeriodicBoundaryConditions())

complex_system.addForce(force1)


True


5

In [53]:
barostat = MonteCarloMembraneBarostat(1*bar, 200*bar*nanometer, 310*kelvin, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree)
complex_system.addForce(barostat)

integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')
#prop = {'OpenCLPrecision':'single', 'OpenCLDeviceIndex':'1'}                                                                                                                                                         
prop = {'CudaPrecision':'single', 'CudaDeviceIndex':'0'}

#simulation = Simulation(complex_structure.topology, complex_system, integrator)
simulation = Simulation(complex_structure.topology, complex_system, integrator, platform, prop)


In [54]:
simulation.context.setPositions(complex_structure.positions)
simulation.minimizeEnergy(maxIterations=150)

In [55]:
##PSF files don't like having numbered atom types, because at 10,000 VMD fails
for a in complex_structure.atoms:
    a.type = '0'
complex_structure.save('output.psf', overwrite=True)
complex_structure.save('output.pdb', overwrite=True)

In [56]:
simulation.reporters.append(DCDReporter('output.dcd', 10000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True, speed=True))
simulation.step(500000)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,-525363.1203514659,279.77046170345267,0
2000,-516064.5381647763,307.03535938673485,140
3000,-517426.7267691549,309.55477524380893,140
4000,-519388.89842413645,309.8427113749023,140
5000,-519296.32303600153,309.7974900649862,140
6000,-521489.914087452,311.6154649075726,141
7000,-522071.36171549745,310.7714141063573,141
8000,-522933.15446023317,310.1383685518029,141
9000,-523504.72164068185,309.0647033402773,141
10000,-524177.0032318812,306.5750873975963,139
11000,-523502.8667846755,311.9026964277646,139
12000,-523476.0584061248,310.4240325622211,139
13000,-525518.5139104882,310.32610495698873,139
14000,-526938.6903345436,310.3007024989715,139
15000,-526294.0178825133,308.53200094574464,139
16000,-526794.8525780034,309.46559430044795,139
17000,-524752.2115396885,310.3995803002364,139
18000,-525746.1708956193,310.2844252799698,139
19000,-527819.1498516044,311.00749732935765,140
20000,-527093.1997679039,308.241823

172000,-550424.8799591167,311.1245969811882,139
173000,-551546.1328916703,311.08768147796343,139
174000,-550451.7242891258,309.26352507361224,139
175000,-549958.5247002733,310.91390227113203,139
176000,-549875.3620126988,312.2121074250472,139
177000,-550072.6995948479,311.77535502100125,139
178000,-550611.3311063191,310.99124785043927,139
179000,-550174.0785655277,310.11600486148444,139
180000,-552415.3697832865,310.2800881928565,139
181000,-550054.1133405585,309.46993648660754,139
182000,-551287.1628508829,306.09138731556925,139
183000,-551816.0631449334,308.280710710839,139
184000,-550809.1532597239,310.95705347371904,139
185000,-550919.6582881925,310.12606138289163,139
186000,-550756.1055557309,310.1001681826944,139
187000,-552257.2729268074,313.51122814866454,139
188000,-551307.2993516149,307.8406499038503,139
189000,-552339.9733036892,310.8194259838305,139
190000,-551767.8763673911,307.778237772794,139
191000,-551096.3739541844,311.6343618980901,139
192000,-549992.979762454,310.37

342000,-549822.4524415899,310.82907648812045,139
343000,-550137.3103363668,311.6371915917688,139
344000,-552568.8530127364,312.033352830864,139
345000,-551388.0840170453,311.1004724386588,139
346000,-551307.8887133794,310.97851802302444,139
347000,-552168.4359083581,310.61927855685155,139
348000,-551520.4407196441,309.99609401911755,139
349000,-551387.0224051429,309.19799176498674,139
350000,-550247.3427948928,310.0153871192442,139
351000,-550223.4180622105,311.3142529448066,139
352000,-551040.3902559606,310.2475876141338,139
353000,-551707.1594899986,308.2932597814013,139
354000,-552197.1876381966,311.15480648769096,139
355000,-551383.5699066725,311.08445617945125,139
356000,-552347.379224163,312.42252886432124,139
357000,-551839.7688799924,309.4527614286648,139
358000,-551211.7743571084,309.52900402786264,139
359000,-550476.5156060467,307.5145264637794,139
360000,-551614.3427822497,312.5002677022878,139
361000,-552961.352432807,311.1617307156464,139
362000,-552129.0117022693,310.0554

In [57]:
with open('solvated_equilibrated_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )
    
with open('solvated_equilibrated_state.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                simulation.context.getState(getPositions=True,
                                     getForces=True, getEnergy=True,
                                     enforcePeriodicBox=True)
            )
    )


In [58]:
waterAtoms = [count for count, i in enumerate(complex_structure.topology.atoms()) if i.name=='O' and i.residue.name=='HOH']

positions = simulation.context.getState(getPositions=True).getPositions()/nanometer
bah = np.array(positions)
print(np.mean(bah[proteinAtoms], axis=0))
boxoffset = (np.max(bah[waterAtoms],axis=0)-np.min(bah[waterAtoms]))/2

proteinCOM = np.mean(bah[proteinAtoms],axis=0)
print(proteinCOM)
bah = bah-proteinCOM
print(np.mean(bah[proteinAtoms],axis=0))
print(boxoffset)
positions = boxoffset+bah
print(np.mean(positions[proteinAtoms],axis=0))

PDBFile.writeFile(simulation.topology, positions*nanometer, open('solvated_equilibrated.pdb', 'w'))

[0.03657776 0.03211967 0.09601887]
[0.03657776 0.03211967 0.09601887]
[1.31104357e-14 1.08979709e-14 2.61507158e-15]
[3.48460174 3.48328448 4.23624969]
[3.48460174 3.48328448 4.23624969]
